<a href="https://colab.research.google.com/github/omargrojas/ADA-2023/blob/main/ds-leadscore-tutorial-activity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Data Science 101 - Lead Scoring Scenario

## Part 1: Problem

Today we are going to look at a very common data science problem found in e-commerce and web based software or services (like Xero!). **Lead scoring!**

### The Sales Process

An education company named X Education sells online courses to industry professionals. On any given day, many professionals who are interested in the courses land on their website and browse for courses.

1. Individuals land on the website, they might browse the courses, watch some videos and if they are interested they will fill up a form for the course.

1. Once individuals fill out a form, they have now become a potential customer or **lead**

1. Employees from the sales team will contact leads by emailing or calling them with the goal of getting the lead to sign up for the course - called a **conversion**

We will be using a consulting model called "SCQA" to help break down the problem.

### Situation

- Currently only 38% of leads contacted go on to become a paying customer.
- The CEO wants to grow the customer base by 20% over the next year

### Complication

- The head of the sales & advertising department has marketing budget to increase advertising and generate 20% more leads, but does not have the headcount to contact every lead already, so more leads would not necessarily mean more conversions
- The department head knows that a conversion rate of 38% means a lot of time and money is wasted on customers who never intended to sign up.

### Question

- How can data science help to convert more leads with the same or less amount of calls?

### Answer

- A lead scoring model trained on historical conversion data could help us to prioritise leads and form a better queue to call from 'hottest' lead to 'coldest'

Your idea for a lead scoring model gets approved - now you need to look at the data you and decide what model you are going to build!

## Part 2: Tools

Today we are going to be using a few different tools in our lead scoring scenario:

### Python

In [ ]:
# some basic python commands

# create an array
x = [1, 2, 3, 4, 5]

In [ ]:
# try the print command print(x)

In [ ]:
# try min(x)

In [ ]:
# try max(x)

`min`, `max` and `print` are what we call **functions**. These are preprogrammed commands to do common calculations on manipulations.

### Jupyter Notebook in Google Colab

Jupyter notebook contains **cells** to run each bit of code at a time

You can run a cell by using `ctrl` + `enter`

Or because we are hosting the notebook in Google Colab you can click the `play` button at the top of the cell

You can add a new text or code cell using the buttons at the top of the notebook

### Importing packages

To do data science we need to add a bit more functionality than is available in the base python code.

We do this by importing packages that have extra functions for data analysis, statistics and machine learning.

The data scientists tool kit generally includes the following packages:

In [ ]:
# package for multi-dimensional arrays and matrices
import numpy as np

# package for data manipulation and analysis
import pandas as pd

# packages for creating plots and graphs
import matplotlib.pyplot as plt
import seaborn as sns

# visulaisation
from matplotlib.pyplot import xticks
%matplotlib inline

# ignoring any warnings for this tutorial and you should absolutely ignore warnings forever wait I mean
import warnings
warnings.filterwarnings("ignore")

We use the `import` function to load the extra functions into our python session.

The `as` just shortens the path name of the function so we can call `pd.function_name()` instead of `pandas.function_name()`

## Part 3: Load Data

We use functions from the `pandas` package to load in our data from a csv. Then it's important to check that we have loaded the data correctly!

Because the datasets are normally too large to parse all at once we can't just browse it like a spreadsheet. We use functions instead to browse the data, looking at the number of records and the content stored in each column.

It's also very useful to look at the type of data in each column and check for any duplicates or missing/Null values. Doing this up front will save us headaches later.

In [ ]:
# read in the data into a pandas dataframe
leads_df = pd.read_csv('https://raw.githubusercontent.com/hjamau/ds-leadscore-tutorial/master/Leads.csv')

In [ ]:
# class of the object
type(leads_df)

In [ ]:
# how many rows and columns
leads_df.shape

In [ ]:
# top 10 rows of the dataframe df.head()
leads_df.head(10)

In [ ]:
# bottom 5 rows of the dataframe try df.tail()

In [ ]:
# describe
leads_df.describe()

In [ ]:
leads_df.describe(include = 'all')

In [ ]:
# column names
leads_df.columns

In [ ]:
# selecting columns
leads_df.Specialization

In [ ]:
# this also selects a column
leads_df['Specialization']

In [ ]:
# We can also subset a dataframe to specific columns
leads_df[['Specialization', 'Converted']]

In [ ]:
# We can apply functions over columns. Try sum() on the Converted column

In [ ]:
# We can do multiple functions in one command
# let’s use the converted column, sum and shape functions to check existing conversion rate in data set

In [ ]:
# Let's use the sum and duplicated functions to do a very important step - check duplicates
leads_df.duplicated()

In [ ]:
# use sum() and dumplicated to get the count of duplicated columns

In [ ]:
# look at null/missing values
leads_df.isnull()

In [ ]:
# use sum() and isnull() to get count of null values accross the columns

In [ ]:
# Use value counts to look at the levels and counts of categories
leads_df.Specialization.value_counts(dropna=False)

In [ ]:
# Try with another column

## Part 4: Data Cleaning

In the real world we often have imperfect or missing data that we need to 'clean' to get ready for data analysis.

Using what we learnt above about the data, we will now apply some changes to the dataframe to fill in any gaps:

In [ ]:
# There are "select" values in columns, that doesn't make any sense for specialization (I know it's a Z not an S its American)
# This is because customer did not select any option from the list and the form was built badly, hence it shows select.

# Change 'Select' values to NaN.
leads_df = leads_df.replace('Select', np.nan)

In [ ]:
# lets look at the specialization column again
leads_df['Specialization'].value_counts(dropna=False)

In [ ]:
# It maybe the case that the lead has not entered any specialization if their option is not availabe on the list
# We can make a category "Others" for missing values.
leads_df['Specialization'] = leads_df['Specialization'].replace(np.nan, 'Other')

In [ ]:
# lets look at the specialization column again
leads_df['Specialization'].value_counts(dropna=False)

## Part 5: Data Exploration

Each row in our data is a single observation made up of a values in each column.

Now we have cleaned the data we can start exploring each of the columns we have as **variables** in the dataset:

- `Prospect ID` is the identifier variable for the data across each row

- `converted`  is our **target** variable (also known as the response variable) i.e. the variable that we are interested in predicting.

    - It is binary and the event either happens or it doesn't i.e. 0 or 1

- We have a mix of 7 categorical and continuous variables we can as **predictor** variables (also called explanatory variable, feature, input variable or independent variable)

We can explore the data to see if there are any correlations between the target and predictor variables.

First for our categorical variables let's look at side by side bar plots:

In [ ]:
# lead Origin
fig, axs = plt.subplots(figsize = (15,7.5))
sns.countplot(x = "LeadOrigin", hue = "Converted", data = leads_df)
xticks(rotation = 90)

In [ ]:
# lead source

In [ ]:
# Specialization

In [ ]:
# Occupation

For our continuous variables we can look at box plots

In [ ]:
# Total Visits
sns.boxplot(y = 'TotalVisits', x = 'Converted', data = leads_df)

In [ ]:
# Total Time Spent on Website

In [ ]:
# Page Views Per Visit

What comments can you make about the different variables and their conversion rates?

## Part 7: Model Building

Now we have a reasonably good idea about what we want to predict and the relationships in each variable we can start to build a predictive model for our lead score.

As we said above we have a binary target that we want to predict and a mix of 7 categorical and continuous variables we can use as predictor variables

Because our target is binary (0 or 1)  - this is a **classification** problem, meaning that we want to predict whether a lead will convert or not.

So we need to pick a model for classification. The first choice for many binary classification problems is a **logistic regression** model because of its simplicity and interpretability.

There are other options like **random forests** or **neural networks** that can provide better accuracy, but depending on the data logistic regression can perform very well!

### Training-testing split

The next part of building a predictive model is to split the data into a **training** set and **testing** set.
This lets us withhold data from the model when training so that we can test it's performance on data it has not seen before - just like it will be doing for our lead scoring in real life.

- Train 80%
- Test 20%

We also need to transform the data to get it ready for modelling
- Remove the ID label (not useful as a predictor)
- Split out the target column
- For categorical variables with multiple levels, create dummy features (one-hot encoded)

In [ ]:
# package for training models
from sklearn import datasets, linear_model, metrics
from sklearn.model_selection import train_test_split

In [ ]:
# vector to store conversion results
y = leads_df['Converted']

In [ ]:
# use the get_dummies variable on Occupation column
pd.get_dummies(leads_df['Occupation'])

In [ ]:
# taking catagorical variables and creating dummy variables
# here we have subsetted the dataframe to only the catagorical columns
dummy1 = pd.get_dummies(leads_df[['LeadOrigin', 'LeadSource', 'Specialization', 'Occupation']], drop_first=True)

In [ ]:
# look at dummies dataframe

In [ ]:
# Adding the results of the dummy variables and remaining continous variables
# in a dataframe with all the predictors
X = pd.concat([leads_df[['TotalVisits', 'TotalTime', 'PageViews']], dummy1], axis=1)

In [ ]:
# look at new X dataframe

In [ ]:
# create training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 404)

In [ ]:
# print the dimensions of X_train, X_test, y_train and y_test

Now we are ready to 'fit' our logistic regresion model to the data

In [ ]:
# fit a model
lm = linear_model.LogisticRegression()
model = lm.fit(X_train, y_train)

Easy as that! Two lines of code.

## Part 8: Model Evaluation

Next is the important bit - how good is our model at predicting conversions?

There are many metrics to evaluate a model and two ways we can get estimates
- 'in-sample' evaluating on our training data
- 'out of sample' evaluating on our testing data

In [ ]:
# in sample accuracy
metrics.accuracy_score(y_train, model.predict(X_train))

In [ ]:
# Get predictions on testing dataset
y_test_pred = model.predict(X_test)

In [ ]:
# Can you do the same for the out of of sample accuracy?

The in-sample estimate always tends to be higher accuracy - which is why we need to evaluate our model on data it has not seen before to get a true estimate of real life performance!

### Confusion Matrix

For classification models we can use a tool called a **confusion matrix** to see how the model performed on classifying non-conversions compared to conversions

In [ ]:
# Confusion matrix
# imput is the actual conversions vs the predicted conversions for each lead in the test set
confusion = metrics.confusion_matrix(y_test, y_test_pred)

confusion

One thing we are interested in is how many leads predicted as converters were actually converters and how many predicted as non-converters were actually non converters:

In [ ]:
# some more things we can calculate from our confusion matrix
TP = confusion[1,1] # true positive
TN = confusion[0,0] # true negatives
FP = confusion[0,1] # false positives
FN = confusion[1,0] # false negatives

print("True positive", TP)
print("True negative", TN)
print("False Positives", FP)
print("False Negatives", FN)

And the proportions of each:

- **Sensitivity** (also called the true positive rate, the recall) measures the proportion of actual positives that are correctly identified as such.

- **Specificity** (also called the true negative rate) measures the proportion of actual negatives that are correctly identified as such.

In [ ]:
# Let's see the sensitivity of our logistic regression model
se = TP / float(TP+FN)
# Let us calculate specificity
sp = TN / float(TN+FP)

# Out of all those who were converters - what proportion did our model get right?
print("Sensitivity", se)
# Out of all those who were non-converters - what proportion did our model get right?
print("Specificity", sp)

The default for predictions is all p > 0.5 are classed as 1 (i.e. conversion) and all p < 0.5 are classed as 0 (non-conversion)

Depending on what is worse
- Mis-classifying a non-converter as a converter and calling them by mistake - low specificity
- Mis-classifying a converter as a non-converter and and not calling them - low sensitivity

We can tune the sensitivity and specificity of our model, by moving this threshold of 0.5 higher or lower.

In [ ]:
# roc curve
y_pred_proba = model.predict_proba(X_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(y_test,  y_pred_proba)
auc = metrics.roc_auc_score(y_test, y_pred_proba)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.xlabel("False Positive Rate (1-Specificity)")
plt.ylabel("True Positive Rate (Sensitivity)")
plt.show()

## Part 9: Interpreting Results & Lift

The great thing about logistic regression is it gives us the predicted probabilities of that observation falling into that class (in our case conversion = 0 or conversion = 1).

We can use this to rank our leads into deciles of the 'hottest' and compare the number of actual conversions in each decile.

Then we can see how this compares to the existing method - or no method.

In [ ]:
# create dataframe
probs = pd.concat([y_test.reset_index(drop = True), pd.Series(y_pred_proba)], axis = 1)
probs.columns = ['actual', 'prob_c']

In [ ]:
# look at probs dataframe

In [ ]:
# cut data into decilies
# i.e. putting the leads into buckets of top 10%, 20%, ... based on how likely they are to convert
probs['deciles'] = pd.qcut(probs.prob_c.rank(method='first'), 10)

In [ ]:
# use value_counts() to look at new decile column

In [ ]:
# compare actual conversion in each decile
# pandas groupby is a useful function for
lift = probs.groupby(probs.deciles)['actual'].agg(["sum", "count"]).reset_index()

# existing conversion rate in test data
x = sum(lift['sum'])/sum(lift['count'])

# calculate conversion probs
lift['prob_con'] = lift['sum']/lift['count']

# get cumulative counts and probabilities
lift['sum_c'] = lift['sum'].iloc[::-1].cumsum()
lift['prop_c'] = lift['sum_c']/731

# old model - 40% conversion by random selection
lift['old'] = lift['count']*x

In [ ]:
# dataframe with our metrics
lift

### Waterfall Plot

This plot shows us for each decile (ordered 'hottest' to 'not hot' leads) how many conversions we would expect to get by contacting leads in each decile compared to when we just pick leads at random that have a 38% conversion rate.

In [ ]:
# Waterfall analysis plot
ax = plt.gca()

ax.bar(lift.index, lift['count'].iloc[::-1], color = 'lightgreen')
ax.bar(lift.index, lift['sum'].iloc[::-1])
ax.plot(lift.index, lift['old'].iloc[::-1], color = 'red')
plt.xticks(lift.index, labels=lift.index+1)
plt.title("Waterfall Analysis")
plt.xlabel("Deciles")
plt.ylabel("Conversions")

plt.show()

We can see that our model puts our more likely converters in the top deciles. This means that the sales team has a way of prioritising leads
1. Top 10% of leads are 98% likely to convert so they should always be at the top of the queue
2. Middle % of leads might still convert but less of a priority
3. Bottom 30% of leads are very unlikely to convert - don't waste time and resource here! Ignore them!

*Next idea - can you work out those in the top decile who might convert on without even needing a sales call? - AB testing*

### Lift plot

Here we compare the cumulative gains in converted customers we get from our model compared to what we get by calling leads randomly.

In [ ]:
# Lift plot
ax = plt.gca()

d = np.linspace(0.1,1,10).round(1)

ax.plot(lift.index, lift['prop_c'].iloc[::-1]*100, marker='o')
ax.plot(lift.index, d*100, color = 'red', marker='o')
plt.xticks(lift.index, labels=d*100)
plt.title("Lift Chart")
plt.xlabel("% of Leads")
plt.ylabel("% of Conversions")

plt.show()

This chart shows us that by contacting just the top 10% of leads we actually capture 24% of all conversions.

This is compared to having no model where we will only get 10% of all conversions for every 10% of leads we contact.

We only need to contact the top 70% leads from our model to capture 95% of all conversions.

This model shows we can convert more leads without making more calls! We just have to call the right people in the right order!

# Part 10: Next steps

You could try improve model accuracy by:
- Try a different model
- Transform variables
- Variable selection
- Tune hyperparameters
- Adding new data

Model building is an iterative process until you reach the level of accuracy required